In [1]:
import pandas as pd
import nltk; nltk.download('stopwords')
nltk.download('punkt')

from nltk.tokenize import word_tokenize 

# Regular Expressions
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# import warnings
# warnings.filterwarnings("ignore",category=DeprecationWarning)

# from ngram import NGram

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\uyent\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\uyent\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
listings = pd.read_csv("listings.csv", usecols=['id', 'name', 'description', 'neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type', 
                                            'price', 'availability_365'])
listings = listings.rename(columns={'id': 'listing_id'})

reviews = pd.read_csv("reviews.csv", usecols=['listing_id', 'comments'])

In [3]:
df = pd.merge(listings, reviews, on='listing_id')

In [4]:
df.describe()

,listing_id,availability_365
count,1.166689e+06,1.166689e+06
mean,1.296548e+07,1.628377e+02
std,1.005105e+07,1.247375e+02
min,3.831000e+03,0.000000e+00
25%,3.768977e+06,4.000000e+01
50%,1.199379e+07,1.500000e+02
75%,2.070862e+07,2.860000e+02
max,3.855316e+07,3.650000e+02


In [5]:
df.head()

,listing_id,name,description,neighbourhood_cleansed,zipcode,property_type,room_type,price,availability_365,comments
0,3831,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Clinton Hill,11238,Guest suite,Entire home/apt,$89.00,192,"lisa is a wonderful, kind and thoughtful host...."
1,3831,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Clinton Hill,11238,Guest suite,Entire home/apt,$89.00,192,The best thing about this place is the locatio...
2,3831,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Clinton Hill,11238,Guest suite,Entire home/apt,$89.00,192,LisaRoxanne was friendly and helpful. The room...
3,3831,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Clinton Hill,11238,Guest suite,Entire home/apt,$89.00,192,LisaRoxanne is very nice and communicative. Th...
4,3831,Cozy Entire Floor of Brownstone,Urban retreat: enjoy 500 s.f. floor in 1899 br...,Clinton Hill,11238,Guest suite,Entire home/apt,$89.00,192,Lisa was a very pleasant and nice host that ac...


In [6]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df['comments'][1:100]))

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['the', 'best', 'thing', 'about', 'this', 'place', 'is', 'the', 'location', 'you', 're', 'right', 'in', 'the', 'middle', 'of', 'clinton', 'hill', 'with', 'easy', 'access', 'to', 'restaurants', 'on', 'franklin', 'avenue', 'and', 'in', 'fort', 'greene', 'you', 're', 'also', 'close', 'to', 'the', 'and', 'trains', 'so', 'transportation', 'is', 'easy', 'the', 'space', 'is', 'clean', 'and', 'neat', 'if', 'spare', 'it', 'really', 'is', 'much', 'more', 'of', 'one', 'floor', 'apartment', 'than', 'private', 'room', 'as', 'you', 'will', 'only', 'have', 'interactions', 'with', 'the', 'family', 'on', 'the', 'shared', 'stairs', 'and', 'stoop', 'there', 'is', 'however', 'no', 'door', 'at', 'the', 'top', 'of', 'the', 'stairs', 'so', 'sounds', 'travel', 'from', 'the', 'rest', 'of', 'the', 'house', 'lisa', 'roxanne', 'was', 'kind', 'and', 'accommodating', 'and', 'the', 'children', 'are', 'adorable']


In [9]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [15]:
a =df.comments[1:100].to_string()
len(a)
a

"1     The best thing about this place is the locatio...\n2     LisaRoxanne was friendly and helpful. The room...\n3     LisaRoxanne is very nice and communicative. Th...\n4     Lisa was a very pleasant and nice host that ac...\n5     Lisa était sympa, toujours prête à aider. L'ét...\n6     The reservation was canceled 5 days before arr...\n7     We were greeted by Lisa's husband at the door....\n8     Lisa and her family were warm and welcoming ho...\n9     The place was great. Lisaroxanne was a great h...\n10    Bien que la chambre soit défraichi, nous avons...\n11    LisaRoxanne met us out on the sidewalk, and wa...\n12    She is so wonder full. We got friendly and hom...\n13    LisaRoxanne and her family are some of the mos...\n14    Lisa was great! Communication prior to the sta...\n15    Han sido unos días muy agradables. La casa est...\n16    LisaRoxanne is very kind and accommodative hos...\n17    Nice apartment, good location. Very near to Su...\n18    We had a very good time 

In [40]:
from nltk.util import ngrams

s= df.comments[1:1000].to_string().lower()
s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
# print
# i=0
# while i < len(s)
#     print('i:', i)


tokens = [token for token in s.split(" ") if token != ""]
test_word= '\n'
if any(test_word in i for i in tokens):
    print('i',i)
    del i
    
print('tk', tokens)
    
output = list(ngrams(tokens, 5))
# print(output, 'output')


# # Remove Stop Words
# data_words_nostops = remove_stopwords(data_words)

# # Form Bigrams
# data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en


nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(output, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# print(data_lemmatized[1:10])

i 1


tk ['1', 'the', 'best', 'thing', 'about', 'this', 'place', 'is', 'the', 'locatio', '\n2', 'lisaroxanne', 'was', 'friendly', 'and', 'helpful', 'the', 'room', '\n3', 'lisaroxanne', 'is', 'very', 'nice', 'and', 'communicative', 'th', '\n4', 'lisa', 'was', 'a', 'very', 'pleasant', 'and', 'nice', 'host', 'that', 'ac', '\n5', 'lisa', 'tait', 'sympa', 'toujours', 'pr', 'te', 'aider', 'l', 't', '\n6', 'the', 'reservation', 'was', 'canceled', '5', 'days', 'before', 'arr', '\n7', 'we', 'were', 'greeted', 'by', 'lisa', 's', 'husband', 'at', 'the', 'door', '\n8', 'lisa', 'and', 'her', 'family', 'were', 'warm', 'and', 'welcoming', 'ho', '\n9', 'the', 'place', 'was', 'great', 'lisaroxanne', 'was', 'a', 'great', 'h', '\n10', 'bien', 'que', 'la', 'chambre', 'soit', 'd', 'fraichi', 'nous', 'avons', '\n11', 'lisaroxanne', 'met', 'us', 'out', 'on', 'the', 'sidewalk', 'and', 'wa', '\n12', 'she', 'is', 'so', 'wonder', 'full', 'we', 'got', 'friendly', 'and', 'hom', '\n13', 'lisaroxanne', 'and', 'her', 'fami

In [27]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1)]]


In [28]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [29]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.001*"ideaal" + 0.001*"ligge" + 0.001*"situate" + 0.001*"situated" + '
  '0.001*"sized" + 0.001*"starter" + 0.001*"ngastvrijheid" + 0.001*"old" + '
  '0.001*"yet" + 0.001*"discreet"'),
 (1,
  '0.898*"host" + 0.000*"wonderful" + 0.000*"family" + 0.000*"amazing" + '
  '0.000*"flexible" + 0.000*"phantastic" + 0.000*"wonder" + 0.000*"fantastic" '
  '+ 0.000*"go" + 0.000*"friend"'),
 (2,
  '0.842*"nice" + 0.000*"family" + 0.000*"really" + 0.000*"hospitable" + '
  '0.000*"people" + 0.000*"confortable" + 0.000*"quiet" + 0.000*"give" + '
  '0.000*"person" + 0.000*"apartment"'),
 (3,
  '0.065*"meet" + 0.001*"never" + 0.001*"shunichi" + 0.001*"actually" + '
  '0.001*"friend" + 0.001*"apartment" + 0.001*"door" + 0.001*"pleasure" + '
  '0.001*"person" + 0.001*"need"'),
 (4,
  '0.447*"welcome" + 0.196*"welcoming" + 0.000*"make" + 0.000*"space" + '
  '0.000*"clean" + 0.000*"where" + 0.000*"mother" + 0.000*"really" + '
  '0.000*"come" + 0.000*"list"'),
 (5,
  '0.924*"very" + 0.000*"apartment

In [30]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -19.725154334626662

Coherence Score:  0.6425320971022934


In [31]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\uyent\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


TypeError: Object of type complex is not JSON serializable

PreparedData(topic_coordinates=                                x                           y  topics  \
topic                                                                   
18       (-0.4246911135347893+0j)    (0.34105861522883424+0j)       1   
5       (-0.24150850612025468+0j)   (-0.16614927286604358+0j)       2   
11      (-0.27081128198490706+0j)    (-0.2623213329352752+0j)       3   
1       (-0.18440677930876415+0j)   (-0.07595756384477645+0j)       4   
2        (-0.0897138535244738+0j)  (-0.014190563381506871+0j)       5   
19      (-0.04031569160471533+0j)   (0.001976793343656978+0j)       6   
8       (-0.06388513656823328+0j)  (-0.004889540464400869+0j)       7   
6      (-0.008456966845280758+0j)   (0.009708214627770245+0j)       8   
17       (0.11434329641569435+0j)    (0.01922102479770672+0j)       9   
7        (0.06327525406478449+0j)   (0.019232915409059373+0j)      10   
4        (0.06460925298231007+0j)   (0.019340250503439863+0j)      11   
3        (0.12123698